In [26]:

import numpy as np
import pandas as pd
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
import tqdm as nootbook_tqdm
from sklearn.ensemble import AdaBoostRegressor


In [27]:
df=pd.read_csv("cloudy_cluster.csv")
df.head()

,GHI,Clearsky DHI,cluster,DHI,DNI,Wind Speed,Relative Humidity,Temperature,Clearsky DNI,Clearsky GHI,Cloud Type,Solar Zenith Angle
0,0,0,1,0,0,5.6,50.05,-5.4,0,0,7,163.33
1,0,0,1,0,0,5.7,49.67,-5.3,0,0,7,162.48
2,0,0,1,0,0,5.8,52.92,-5.1,0,0,8,159.58
3,0,0,1,0,0,6.0,52.45,-5.0,0,0,0,155.37
4,0,0,1,0,0,6.1,55.58,-4.9,0,0,0,150.41


In [28]:
df.shape

(12635, 12)

In [29]:
df.isnull().sum()

GHI                   0
Clearsky DHI          0
cluster               0
DHI                   0
DNI                   0
Wind Speed            0
Relative Humidity     0
Temperature           0
Clearsky DNI          0
Clearsky GHI          0
Cloud Type            0
Solar Zenith Angle    0
dtype: int64

In [30]:
df.drop(['cluster'], axis = 1, inplace = True) 

In [31]:
df.describe()

,GHI,Clearsky DHI,DHI,DNI,Wind Speed,Relative Humidity,Temperature,Clearsky DNI,Clearsky GHI,Cloud Type,Solar Zenith Angle
count,12635.000000,12635.000000,12635.000000,12635.000000,12635.000000,12635.000000,12635.000000,12635.000000,12635.000000,12635.000000,12635.000000
mean,45.259201,18.276138,23.818283,90.733043,2.091619,55.478055,4.675797,191.764306,83.672893,2.693550,105.951741
std,85.973604,35.184072,48.906084,203.054985,1.157924,17.844177,9.852023,316.693066,177.238869,3.077329,28.252325
min,0.000000,0.000000,0.000000,0.000000,0.200000,7.910000,-19.500000,0.000000,0.000000,0.000000,16.250000
25%,0.000000,0.000000,0.000000,0.000000,1.300000,42.360000,-3.400000,0.000000,0.000000,0.000000,84.465000
50%,0.000000,0.000000,0.000000,0.000000,1.800000,56.690000,3.400000,0.000000,0.000000,0.000000,107.280000
75%,46.000000,29.000000,29.000000,29.000000,2.700000,67.660000,13.000000,359.000000,71.000000,6.000000,126.155000
max,332.000000,339.000000,295.000000,949.000000,8.800000,98.910000,32.100000,1021.000000,1025.000000,9.000000,163.760000


In [32]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)

In [33]:
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['GHI']])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42,shuffle=False)

In [35]:
model = AdaBoostRegressor(n_estimators=100,learning_rate=0.1)
model.fit(X_train, y_train)
print(model)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


AdaBoostRegressor(learning_rate=0.1, n_estimators=100)


In [36]:
import time
start=time.time()
prediction = model.predict(X_test)
end=time.time()
continental=end-start
print("inference time:",continental)

inference time: 0.07203936576843262


In [37]:
prediction.shape

(3791,)

In [38]:
prediction = prediction.reshape(prediction.shape[0], 1)

In [39]:
prediction.shape

(3791, 1)

In [40]:
Pred= s2.inverse_transform(prediction)

In [41]:
Actual= s2.inverse_transform(y_test)

In [42]:
Calculated = pd.DataFrame(Pred, columns = ['Prediction'])

In [43]:
Calculated.to_csv(r'E:\CatBoost_1\Golden\AdaBoost\cloudy hours\prediction.csv', index = False)

In [44]:
Actual = pd.DataFrame(Actual, columns = ['Actual'])

In [45]:
Actual.to_csv(r'E:\CatBoost_1\Golden\AdaBoost\cloudy hours\Actual.csv', index = False)

In [46]:
x=Pred
z=Actual

In [47]:
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE


print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)


Root Mean Square Error: 4.080265867849509
Mean Square Error: 16.648569552337708
Mean Absolute Error: 2.9272120081570128


In [48]:
max= x.max()
max

309.2187050359701

In [49]:
min=x.min()
min

1.7775634844101604

In [50]:
NRMSE= (RMSE/(max-min))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 1.3271697623999426
